# Stage 2: Fast Model Training with Pre-computed Embeddings

**Purpose**: Rapid experimentation with different fusion methods and model architectures.

## Features:
- ✅ **Auto-checkpoint**: Saves best model + progress every epoch
- ✅ **Resume training**: Continue from last checkpoint if interrupted
- ✅ **Live metrics**: Loss, R², MAE, RMSE displayed per epoch
- ✅ **Memory monitoring**: GPU/RAM/Disk usage tracked
- ✅ **Optuna integration**: Hyperparameter optimization with checkpoint

## Available Fusion Methods:
1. **concat**: Simple concatenation
2. **bilinear**: Pairwise feature interactions
3. **cross_attention**: Attend between antibody and antigen
4. **gated**: Learned importance weights
5. **tensor**: Low-rank tensor product

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ════════════════════════════════════════════════════════════════════════════

CONFIG = {
    # ===== PATHS =====
    'base_dir': '/content/drive/MyDrive/AbAg_Project',
    'embeddings_dir': 'embeddings',
    'output_dir': 'models',
    'checkpoint_dir': 'checkpoints',
    
    # Embedding files (from Stage 1)
    'train_embeddings': 'train_embeddings_IgT5_ProtT5-Half_lora_last.npz',
    'val_embeddings': 'val_embeddings_IgT5_ProtT5-Half_lora_last.npz',
    'test_embeddings': 'test_embeddings_IgT5_ProtT5-Half_lora_last.npz',
    
    # ===== MODEL ARCHITECTURE =====
    'fusion_method': 'cross_attention',  # 'concat', 'bilinear', 'cross_attention', 'gated', 'tensor'
    'head_type': 'residual_mlp',         # 'simple_mlp', 'residual_mlp', 'deep_mlp', 'multi_task'
    
    # Fusion settings
    'fusion_hidden_dim': 512,
    'fusion_num_heads': 8,
    'fusion_dropout': 0.1,
    
    # Head settings
    'head_hidden_dims': [512, 256, 128],
    'head_dropout': 0.2,
    'use_batch_norm': True,
    
    # ===== TRAINING =====
    'batch_size': 256,
    'learning_rate': 1e-3,
    'weight_decay': 1e-4,
    'num_epochs': 100,
    'early_stopping_patience': 15,
    'scheduler': 'cosine',
    
    # Loss
    'loss_type': 'mse',
    'huber_delta': 1.0,
    
    # Multi-task (if head_type == 'multi_task')
    'classification_weight': 0.3,
    'regression_weight': 0.7,
    
    # ===== CHECKPOINTING =====
    'checkpoint_every_n_epochs': 1,
    'resume_from_checkpoint': True,
    
    # ===== OPTUNA =====
    'use_optuna': False,
    'optuna_trials': 50,
    
    # ===== MEMORY =====
    'min_disk_space_gb': 2,
    
    # ===== EXPERIMENT =====
    'experiment_name': 'stage2_experiment',
}

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# SETUP
# ════════════════════════════════════════════════════════════════════════════

from google.colab import drive
drive.mount('/content/drive')

!pip install -q optuna psutil

import os
import gc
import json
import time
import shutil
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, OneCycleLR
import optuna
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error
from datetime import datetime
import psutil

# Paths
BASE_DIR = CONFIG['base_dir']
EMBEDDINGS_DIR = os.path.join(BASE_DIR, CONFIG['embeddings_dir'])
OUTPUT_DIR = os.path.join(BASE_DIR, CONFIG['output_dir'])
CHECKPOINT_DIR = os.path.join(BASE_DIR, CONFIG['checkpoint_dir'])

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# RESOURCE MONITOR
# ════════════════════════════════════════════════════════════════════════════

class ResourceMonitor:
    def __init__(self, min_disk_gb=2):
        self.min_disk_gb = min_disk_gb
    
    def get_gpu_memory(self):
        if torch.cuda.is_available():
            return {
                'allocated': torch.cuda.memory_allocated() / 1e9,
                'total': torch.cuda.get_device_properties(0).total_memory / 1e9
            }
        return {'allocated': 0, 'total': 0}
    
    def get_ram_usage(self):
        mem = psutil.virtual_memory()
        return {'used': mem.used / 1e9, 'total': mem.total / 1e9, 'percent': mem.percent}
    
    def get_disk_space(self, path='/content/drive'):
        try:
            usage = shutil.disk_usage(path)
            return {'free': usage.free / 1e9, 'total': usage.total / 1e9}
        except:
            return {'free': 999, 'total': 999}
    
    def print_status(self, prefix=""):
        gpu = self.get_gpu_memory()
        ram = self.get_ram_usage()
        disk = self.get_disk_space()
        print(f"{prefix}GPU: {gpu['allocated']:.1f}/{gpu['total']:.1f}GB | "
              f"RAM: {ram['used']:.1f}/{ram['total']:.1f}GB ({ram['percent']:.0f}%) | "
              f"Disk: {disk['free']:.1f}GB free")
        if disk['free'] < self.min_disk_gb:
            print(f"⚠️ WARNING: Low disk space!")
    
    def clear_memory(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

monitor = ResourceMonitor(CONFIG['min_disk_space_gb'])
monitor.print_status("Initial: ")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# CHECKPOINT MANAGER
# ════════════════════════════════════════════════════════════════════════════

class CheckpointManager:
    def __init__(self, checkpoint_dir, experiment_name):
        self.checkpoint_dir = checkpoint_dir
        self.experiment_name = experiment_name
    
    def get_checkpoint_path(self):
        return os.path.join(self.checkpoint_dir, f"{self.experiment_name}_training.pth")
    
    def get_best_model_path(self):
        return os.path.join(self.checkpoint_dir, f"{self.experiment_name}_best.pth")
    
    def save_checkpoint(self, epoch, model, optimizer, scheduler, history, best_metrics, config):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'history': history,
            'best_metrics': best_metrics,
            'config': config,
            'timestamp': datetime.now().isoformat(),
        }
        torch.save(checkpoint, self.get_checkpoint_path())
        print(f"💾 Checkpoint saved: epoch {epoch+1}")
    
    def save_best_model(self, model, config, metrics, ab_dim, ag_dim):
        torch.save({
            'model_state_dict': model.state_dict(),
            'config': config,
            'metrics': metrics,
            'ab_dim': ab_dim,
            'ag_dim': ag_dim,
            'timestamp': datetime.now().isoformat(),
        }, self.get_best_model_path())
        print(f"⭐ Best model saved: R²={metrics['r2']:.4f}")
    
    def load_checkpoint(self, model, optimizer, scheduler=None):
        path = self.get_checkpoint_path()
        if os.path.exists(path):
            # PyTorch 2.6+ requires weights_only=False for checkpoints with numpy/custom objects
            checkpoint = torch.load(path, weights_only=False)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            if scheduler and checkpoint['scheduler_state_dict']:
                scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            print(f"✅ Resumed from epoch {checkpoint['epoch']+1}")
            return checkpoint
        return None
    
    def clear(self):
        for path in [self.get_checkpoint_path(), self.get_best_model_path()]:
            if os.path.exists(path):
                os.remove(path)
                print(f"🗑️ Removed: {os.path.basename(path)}")

ckpt_manager = CheckpointManager(CHECKPOINT_DIR, CONFIG['experiment_name'])

## 1. Load Embeddings

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, path):
        data = np.load(path)
        self.ab_emb = torch.tensor(data['antibody_embeddings'], dtype=torch.float32)
        self.ag_emb = torch.tensor(data['antigen_embeddings'], dtype=torch.float32)
        self.labels = torch.tensor(data['labels'], dtype=torch.float32)
        self.class_labels = (self.labels > 7.0).long()
        print(f"  Loaded {len(self):,} samples | Ab dim: {self.ab_emb.shape[-1]} | Ag dim: {self.ag_emb.shape[-1]}")
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'antibody_emb': self.ab_emb[idx],
            'antigen_emb': self.ag_emb[idx],
            'label': self.labels[idx],
            'class_label': self.class_labels[idx]
        }

print("Loading embeddings...")
print("Train:")
train_dataset = EmbeddingDataset(os.path.join(EMBEDDINGS_DIR, CONFIG['train_embeddings']))
print("Val:")
val_dataset = EmbeddingDataset(os.path.join(EMBEDDINGS_DIR, CONFIG['val_embeddings']))
print("Test:")
test_dataset = EmbeddingDataset(os.path.join(EMBEDDINGS_DIR, CONFIG['test_embeddings']))

ab_dim = train_dataset.ab_emb.shape[-1]
ag_dim = train_dataset.ag_emb.shape[-1]
print(f"\nDimensions: Ab={ab_dim}, Ag={ag_dim}")

## 2. Model Architecture

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# FUSION MODULES
# ════════════════════════════════════════════════════════════════════════════

class ConcatFusion(nn.Module):
    def __init__(self, ab_dim, ag_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Linear(ab_dim + ag_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.output_dim = hidden_dim
    
    def forward(self, ab, ag):
        return self.proj(torch.cat([ab, ag], dim=-1))

class BilinearFusion(nn.Module):
    def __init__(self, ab_dim, ag_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.ab_proj = nn.Linear(ab_dim, hidden_dim)
        self.ag_proj = nn.Linear(ag_dim, hidden_dim)
        self.bilinear = nn.Bilinear(hidden_dim, hidden_dim, hidden_dim)
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.output_dim = hidden_dim
    
    def forward(self, ab, ag):
        return self.dropout(F.gelu(self.norm(self.bilinear(self.ab_proj(ab), self.ag_proj(ag)))))

class CrossAttentionFusion(nn.Module):
    def __init__(self, ab_dim, ag_dim, hidden_dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.ab_proj = nn.Linear(ab_dim, hidden_dim)
        self.ag_proj = nn.Linear(ag_dim, hidden_dim)
        self.cross_attn_ab = nn.MultiheadAttention(hidden_dim, num_heads, dropout, batch_first=True)
        self.cross_attn_ag = nn.MultiheadAttention(hidden_dim, num_heads, dropout, batch_first=True)
        self.combine = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.output_dim = hidden_dim
    
    def forward(self, ab, ag):
        ab_p = self.ab_proj(ab).unsqueeze(1)
        ag_p = self.ag_proj(ag).unsqueeze(1)
        ab_att, _ = self.cross_attn_ab(ab_p, ag_p, ag_p)
        ag_att, _ = self.cross_attn_ag(ag_p, ab_p, ab_p)
        return self.combine(torch.cat([ab_att.squeeze(1), ag_att.squeeze(1)], dim=-1))

class GatedFusion(nn.Module):
    def __init__(self, ab_dim, ag_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.ab_proj = nn.Linear(ab_dim, hidden_dim)
        self.ag_proj = nn.Linear(ag_dim, hidden_dim)
        self.gate = nn.Sequential(nn.Linear(hidden_dim * 2, hidden_dim), nn.Sigmoid())
        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.output_dim = hidden_dim
    
    def forward(self, ab, ag):
        ab_p, ag_p = self.ab_proj(ab), self.ag_proj(ag)
        g = self.gate(torch.cat([ab_p, ag_p], dim=-1))
        return self.dropout(F.gelu(self.norm(g * ab_p + (1 - g) * ag_p)))

class TensorFusion(nn.Module):
    def __init__(self, ab_dim, ag_dim, hidden_dim, rank=64, dropout=0.1):
        super().__init__()
        self.ab_factor = nn.Linear(ab_dim, rank)
        self.ag_factor = nn.Linear(ag_dim, rank)
        self.output_proj = nn.Sequential(
            nn.Linear(rank, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.output_dim = hidden_dim
    
    def forward(self, ab, ag):
        return self.output_proj(self.ab_factor(ab) * self.ag_factor(ag))

def get_fusion(fusion_type, ab_dim, ag_dim, config):
    h = config.get('fusion_hidden_dim', 512)
    d = config.get('fusion_dropout', 0.1)
    n = config.get('fusion_num_heads', 8)
    
    if fusion_type == 'concat': return ConcatFusion(ab_dim, ag_dim, h, d)
    elif fusion_type == 'bilinear': return BilinearFusion(ab_dim, ag_dim, h, d)
    elif fusion_type == 'cross_attention': return CrossAttentionFusion(ab_dim, ag_dim, h, n, d)
    elif fusion_type == 'gated': return GatedFusion(ab_dim, ag_dim, h, d)
    elif fusion_type == 'tensor': return TensorFusion(ab_dim, ag_dim, h, dropout=d)
    else: raise ValueError(f"Unknown fusion: {fusion_type}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# HEAD MODULES
# ════════════════════════════════════════════════════════════════════════════

class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout=0.2, use_bn=False):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims:
            layers.extend([nn.Linear(prev, h), nn.BatchNorm1d(h) if use_bn else nn.Identity(), nn.GELU(), nn.Dropout(dropout)])
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x).squeeze(-1)

class ResidualMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout=0.2, use_bn=True):
        super().__init__()
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.BatchNorm1d(hidden_dims[0]) if use_bn else nn.Identity(),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.blocks = nn.ModuleList()
        prev = hidden_dims[0]
        for h in hidden_dims[1:]:
            self.blocks.append(nn.Sequential(
                nn.Linear(prev, h),
                nn.BatchNorm1d(h) if use_bn else nn.Identity(),
                nn.GELU(),
                nn.Dropout(dropout)
            ))
            prev = h
        self.output = nn.Linear(prev, 1)
    
    def forward(self, x):
        x = self.input_proj(x)
        for block in self.blocks:
            x = block(x)
        return self.output(x).squeeze(-1)

class MultiTaskHead(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout=0.2, use_bn=True):
        super().__init__()
        layers = []
        prev = input_dim
        for h in hidden_dims[:-1]:
            layers.extend([nn.Linear(prev, h), nn.BatchNorm1d(h) if use_bn else nn.Identity(), nn.GELU(), nn.Dropout(dropout)])
            prev = h
        self.shared = nn.Sequential(*layers)
        self.reg_head = nn.Sequential(nn.Linear(prev, hidden_dims[-1]), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dims[-1], 1))
        self.cls_head = nn.Sequential(nn.Linear(prev, hidden_dims[-1]), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dims[-1], 1))
    
    def forward(self, x):
        s = self.shared(x)
        return self.reg_head(s).squeeze(-1), self.cls_head(s).squeeze(-1)

def get_head(head_type, input_dim, config):
    h = config.get('head_hidden_dims', [512, 256, 128])
    d = config.get('head_dropout', 0.2)
    bn = config.get('use_batch_norm', True)
    
    if head_type == 'simple_mlp': return SimpleMLP(input_dim, h, d, bn)
    elif head_type == 'residual_mlp': return ResidualMLP(input_dim, h, d, bn)
    elif head_type == 'deep_mlp': return SimpleMLP(input_dim, h + h, d, bn)
    elif head_type == 'multi_task': return MultiTaskHead(input_dim, h, d, bn)
    else: raise ValueError(f"Unknown head: {head_type}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# COMPLETE MODEL
# ════════════════════════════════════════════════════════════════════════════

class AffinityPredictor(nn.Module):
    def __init__(self, ab_dim, ag_dim, config):
        super().__init__()
        self.fusion = get_fusion(config['fusion_method'], ab_dim, ag_dim, config)
        self.head = get_head(config['head_type'], self.fusion.output_dim, config)
        self.is_multi_task = config['head_type'] == 'multi_task'
    
    def forward(self, ab, ag):
        return self.head(self.fusion(ab, ag))

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Create model
model = AffinityPredictor(ab_dim, ag_dim, CONFIG).to(device)
print(f"\nModel: {CONFIG['fusion_method']} + {CONFIG['head_type']}")
print(f"Parameters: {count_params(model):,}")

## 3. Training with Checkpoints

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# TRAINING FUNCTIONS
# ════════════════════════════════════════════════════════════════════════════

def get_loss_fn(config):
    t = config.get('loss_type', 'mse')
    if t == 'mse': return nn.MSELoss()
    elif t == 'huber': return nn.HuberLoss(delta=config.get('huber_delta', 1.0))
    else: 
        mse, mae = nn.MSELoss(), nn.L1Loss()
        return lambda p, t: 0.5 * mse(p, t) + 0.5 * mae(p, t)

@torch.no_grad()
def evaluate(model, loader, loss_fn, config, device):
    model.eval()
    total_loss, preds, labels = 0, [], []
    is_mt = config['head_type'] == 'multi_task'
    bce = nn.BCEWithLogitsLoss()
    
    for batch in loader:
        ab, ag = batch['antibody_emb'].to(device), batch['antigen_emb'].to(device)
        y, yc = batch['label'].to(device), batch['class_label'].float().to(device)
        
        out = model(ab, ag)
        if is_mt:
            reg, cls = out
            loss = 0.7 * loss_fn(reg, y) + 0.3 * bce(cls, yc)
            preds.extend(reg.cpu().numpy())
        else:
            loss = loss_fn(out, y)
            preds.extend(out.cpu().numpy())
        
        labels.extend(y.cpu().numpy())
        total_loss += loss.item()
    
    preds, labels = np.array(preds), np.array(labels)
    return {
        'loss': total_loss / len(loader),
        'r2': r2_score(labels, preds),
        'mae': mean_absolute_error(labels, preds),
        'rmse': np.sqrt(np.mean((preds - labels) ** 2))
    }

def train_with_checkpoints(model, train_loader, val_loader, config, device, ckpt_manager):
    """Training loop with checkpointing and resumption."""
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    
    # Scheduler
    if config['scheduler'] == 'cosine':
        scheduler = CosineAnnealingLR(optimizer, T_max=config['num_epochs'])
    elif config['scheduler'] == 'reduce_on_plateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)
    else:
        scheduler = None
    
    loss_fn = get_loss_fn(config)
    is_mt = config['head_type'] == 'multi_task'
    bce = nn.BCEWithLogitsLoss()
    
    # Initialize
    history = {'train_loss': [], 'val_loss': [], 'val_r2': [], 'val_mae': []}
    best_val_loss = float('inf')
    best_metrics = None
    patience_counter = 0
    start_epoch = 0
    
    # Resume if exists
    if config['resume_from_checkpoint']:
        ckpt = ckpt_manager.load_checkpoint(model, optimizer, scheduler)
        if ckpt:
            start_epoch = ckpt['epoch'] + 1
            history = ckpt['history']
            best_metrics = ckpt['best_metrics']
            if best_metrics:
                best_val_loss = best_metrics.get('loss', float('inf'))
    
    print(f"\n{'='*60}")
    print(f"TRAINING: {config['num_epochs']} epochs (starting from {start_epoch+1})")
    print(f"{'='*60}")
    
    for epoch in range(start_epoch, config['num_epochs']):
        model.train()
        train_loss = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']}")
        for batch in pbar:
            ab, ag = batch['antibody_emb'].to(device), batch['antigen_emb'].to(device)
            y, yc = batch['label'].to(device), batch['class_label'].float().to(device)
            
            optimizer.zero_grad()
            out = model(ab, ag)
            
            if is_mt:
                reg, cls = out
                loss = 0.7 * loss_fn(reg, y) + 0.3 * bce(cls, yc)
            else:
                loss = loss_fn(out, y)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            train_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        train_loss /= len(train_loader)
        
        # Validation
        val_metrics = evaluate(model, val_loader, loss_fn, config, device)
        
        # Update scheduler
        if scheduler:
            if config['scheduler'] == 'reduce_on_plateau':
                scheduler.step(val_metrics['loss'])
            else:
                scheduler.step()
        
        # Record history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_metrics['loss'])
        history['val_r2'].append(val_metrics['r2'])
        history['val_mae'].append(val_metrics['mae'])
        
        # Print epoch summary
        print(f"  Train Loss: {train_loss:.4f} | Val Loss: {val_metrics['loss']:.4f} | "
              f"R²: {val_metrics['r2']:.4f} | MAE: {val_metrics['mae']:.4f}")
        
        # Check for improvement
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_metrics = val_metrics
            patience_counter = 0
            ckpt_manager.save_best_model(model, config, val_metrics, ab_dim, ag_dim)
        else:
            patience_counter += 1
        
        # Save checkpoint every N epochs
        if (epoch + 1) % config['checkpoint_every_n_epochs'] == 0:
            ckpt_manager.save_checkpoint(epoch, model, optimizer, scheduler, history, best_metrics, config)
        
        # Early stopping
        if patience_counter >= config['early_stopping_patience']:
            print(f"\n⏹️ Early stopping at epoch {epoch+1}")
            break
        
        monitor.clear_memory()
    
    # Load best model (PyTorch 2.6+ requires weights_only=False)
    best_path = ckpt_manager.get_best_model_path()
    if os.path.exists(best_path):
        model.load_state_dict(torch.load(best_path, weights_only=False)['model_state_dict'])
        print(f"\n✅ Loaded best model (R²: {best_metrics['r2']:.4f})")
    
    return model, history, best_metrics

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# RUN TRAINING (without Optuna)
# ════════════════════════════════════════════════════════════════════════════

if not CONFIG['use_optuna']:
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    
    # Train
    model = AffinityPredictor(ab_dim, ag_dim, CONFIG).to(device)
    model, history, best_metrics = train_with_checkpoints(
        model, train_loader, val_loader, CONFIG, device, ckpt_manager
    )
    
    # Test evaluation
    loss_fn = get_loss_fn(CONFIG)
    test_metrics = evaluate(model, test_loader, loss_fn, CONFIG, device)
    
    print("\n" + "="*60)
    print("FINAL RESULTS")
    print("="*60)
    print(f"Validation: R²={best_metrics['r2']:.4f}, MAE={best_metrics['mae']:.4f}, RMSE={best_metrics['rmse']:.4f}")
    print(f"Test:       R²={test_metrics['r2']:.4f}, MAE={test_metrics['mae']:.4f}, RMSE={test_metrics['rmse']:.4f}")
    monitor.print_status("\nFinal resources: ")
else:
    print("Optuna enabled. Run next cells for hyperparameter optimization.")

## 4. Optuna Optimization (with checkpoints)

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# OPTUNA WITH CHECKPOINT
# ════════════════════════════════════════════════════════════════════════════

def optuna_objective(trial, train_dataset, val_dataset, ab_dim, ag_dim, device):
    config = {
        'fusion_method': trial.suggest_categorical('fusion_method', ['concat', 'bilinear', 'cross_attention', 'gated', 'tensor']),
        'fusion_hidden_dim': trial.suggest_categorical('fusion_hidden_dim', [256, 512, 768]),
        'fusion_num_heads': trial.suggest_categorical('fusion_num_heads', [4, 8]),
        'fusion_dropout': trial.suggest_float('fusion_dropout', 0.05, 0.3),
        'head_type': trial.suggest_categorical('head_type', ['simple_mlp', 'residual_mlp', 'multi_task']),
        'head_hidden_dims': trial.suggest_categorical('head_hidden_dims', [[512, 256, 128], [512, 256], [256, 128]]),
        'head_dropout': trial.suggest_float('head_dropout', 0.1, 0.4),
        'use_batch_norm': trial.suggest_categorical('use_batch_norm', [True, False]),
        'batch_size': trial.suggest_categorical('batch_size', [128, 256, 512]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True),
        'scheduler': trial.suggest_categorical('scheduler', ['cosine', 'reduce_on_plateau']),
        'loss_type': trial.suggest_categorical('loss_type', ['mse', 'huber']),
        'huber_delta': 1.0,
        'num_epochs': 30,
        'early_stopping_patience': 8,
        'checkpoint_every_n_epochs': 100,  # Don't checkpoint during trials
        'resume_from_checkpoint': False,
    }
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    
    model = AffinityPredictor(ab_dim, ag_dim, config).to(device)
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = CosineAnnealingLR(optimizer, T_max=config['num_epochs']) if config['scheduler'] == 'cosine' else None
    loss_fn = get_loss_fn(config)
    is_mt = config['head_type'] == 'multi_task'
    bce = nn.BCEWithLogitsLoss()
    
    best_r2 = -float('inf')
    patience = 0
    
    for epoch in range(config['num_epochs']):
        model.train()
        for batch in train_loader:
            ab, ag = batch['antibody_emb'].to(device), batch['antigen_emb'].to(device)
            y, yc = batch['label'].to(device), batch['class_label'].float().to(device)
            
            optimizer.zero_grad()
            out = model(ab, ag)
            loss = 0.7 * loss_fn(out[0], y) + 0.3 * bce(out[1], yc) if is_mt else loss_fn(out, y)
            loss.backward()
            optimizer.step()
        
        if scheduler: scheduler.step()
        
        val_metrics = evaluate(model, val_loader, loss_fn, config, device)
        trial.report(val_metrics['r2'], epoch)
        
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        if val_metrics['r2'] > best_r2:
            best_r2 = val_metrics['r2']
            patience = 0
        else:
            patience += 1
            if patience >= config['early_stopping_patience']:
                break
        
        monitor.clear_memory()
    
    return best_r2

if CONFIG['use_optuna']:
    print(f"Starting Optuna: {CONFIG['optuna_trials']} trials")
    
    # Create/load study
    study_path = os.path.join(CHECKPOINT_DIR, f"{CONFIG['experiment_name']}_optuna.pkl")
    if os.path.exists(study_path) and CONFIG['resume_from_checkpoint']:
        with open(study_path, 'rb') as f:
            study = pickle.load(f)
        print(f"✅ Resumed study with {len(study.trials)} completed trials")
    else:
        study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
    
    # Run optimization
    remaining_trials = CONFIG['optuna_trials'] - len(study.trials)
    if remaining_trials > 0:
        study.optimize(
            lambda trial: optuna_objective(trial, train_dataset, val_dataset, ab_dim, ag_dim, device),
            n_trials=remaining_trials,
            show_progress_bar=True,
            callbacks=[lambda study, trial: pickle.dump(study, open(study_path, 'wb'))]  # Save after each trial
        )
    
    print(f"\nBest R²: {study.best_trial.value:.4f}")
    print("Best params:")
    for k, v in study.best_trial.params.items():
        print(f"  {k}: {v}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# TRAIN FINAL MODEL WITH BEST PARAMS
# ════════════════════════════════════════════════════════════════════════════

if CONFIG['use_optuna']:
    final_config = {
        **study.best_trial.params,
        'huber_delta': 1.0,
        'num_epochs': 100,
        'early_stopping_patience': 15,
        'checkpoint_every_n_epochs': 1,
        'resume_from_checkpoint': True,
    }
    
    train_loader = DataLoader(train_dataset, batch_size=final_config['batch_size'], shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=final_config['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=final_config['batch_size'], shuffle=False, num_workers=2, pin_memory=True)
    
    final_ckpt = CheckpointManager(CHECKPOINT_DIR, f"{CONFIG['experiment_name']}_final")
    final_model = AffinityPredictor(ab_dim, ag_dim, final_config).to(device)
    
    final_model, history, best_metrics = train_with_checkpoints(
        final_model, train_loader, val_loader, final_config, device, final_ckpt
    )
    
    loss_fn = get_loss_fn(final_config)
    test_metrics = evaluate(final_model, test_loader, loss_fn, final_config, device)
    
    print("\n" + "="*60)
    print("FINAL MODEL RESULTS")
    print("="*60)
    print(f"Validation: R²={best_metrics['r2']:.4f}, MAE={best_metrics['mae']:.4f}")
    print(f"Test:       R²={test_metrics['r2']:.4f}, MAE={test_metrics['mae']:.4f}")

## 5. Save & Visualize

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# SAVE FINAL MODEL
# ════════════════════════════════════════════════════════════════════════════

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_path = os.path.join(OUTPUT_DIR, f"{CONFIG['experiment_name']}_{timestamp}.pth")

model_to_save = final_model if CONFIG['use_optuna'] else model
config_to_save = final_config if CONFIG['use_optuna'] else CONFIG

torch.save({
    'model_state_dict': model_to_save.state_dict(),
    'config': config_to_save,
    'ab_dim': ab_dim,
    'ag_dim': ag_dim,
    'val_metrics': best_metrics,
    'test_metrics': test_metrics,
    'history': history,
}, final_path)

print(f"✅ Final model saved: {final_path}")
print(f"   R² (test): {test_metrics['r2']:.4f}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
# VISUALIZATION
# ════════════════════════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['val_r2'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('R²')
axes[1].set_title(f'Validation R² (best: {max(history["val_r2"]):.4f})')
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['val_mae'])
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('MAE')
axes[2].set_title(f'Validation MAE (best: {min(history["val_mae"]):.4f})')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, f"{CONFIG['experiment_name']}_{timestamp}_curves.png"), dpi=150)
plt.show()

print(f"\n📊 Training curves saved!")

In [ ]:
# Cleanup checkpoints after successful completion
cleanup = input("Clear training checkpoints? (y/n): ").lower() == 'y'
if cleanup:
    ckpt_manager.clear()
    if CONFIG['use_optuna']:
        final_ckpt.clear()
    print("✅ Checkpoints cleared")

## Summary

### Checkpoint Features:
- **Auto-save**: Best model saved whenever validation improves
- **Epoch checkpoints**: Full training state saved every epoch
- **Resume training**: If Colab disconnects, re-run cells to continue
- **Optuna checkpoint**: Study saves after each trial

### Files saved to Google Drive:
```
AbAg_Project/
├── checkpoints/
│   ├── stage2_experiment_training.pth   # Training checkpoint
│   ├── stage2_experiment_best.pth       # Best model
│   └── stage2_experiment_optuna.pkl     # Optuna study
└── models/
    └── stage2_experiment_YYYYMMDD.pth   # Final model
```